### For a comprehensive list of LTCs in Alberta, according to Open Alberta
---

In [32]:
import numpy as np
import pandas as pd
import requests
import tabula

In [53]:
# Read remote pdf into list of DataFrame
file = 'https://open.alberta.ca/dataset/7f4fc1ef-779c-4ebb-a557-d7afc405c826/resource/0a431494-4c96-40c6-bc55-0141b3c72e9e/download/cc-list-funded-sla-ltc.pdf'

# Select page range
pgs = [str(i) for i in range(3,47)]
df = tabula.read_pdf(file, lattice=True, pages=pgs, pandas_options={'header': True})



Got stderr: Sep. 08, 2020 6:41:05 P.M. org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial are not implemented in PDFBox and will be ignored



In [54]:
# A list of pd DFs
dftotal = pd.concat(df, ignore_index=True)

# Clean some text
dftotal = dftotal.apply(lambda x: x.str.replace('\\r',' '))

# Rename funky cols
dftotal.rename(columns = {'Supportive\rLiving 3': 'DSL-3', 
                         'Supportive\rLiving 4' : 'DSL-4',
                         'Supportive\rLiving 4 -\rDementia':'DSL-4-Dementia',
                         'Auxiliary\rHospital' : 'LTC-Auxilary Hospital',
                         'Nursing Home' : 'LTC-Nursing Home',
                         'Operator\rName': 'Operator Name'}, inplace = True)

# Remove rows that describe supportive living cols
# len(dftotal) = 375, 42 rows removed
dftotal = dftotal[dftotal['DSL-3'] != "Designated Supportive Living (Check Mark)"]


In [55]:
# Shift row information 1 column right where Zone is not null 
# (no zones were accurately imported, any that are not null are actually the city)
dftotal[dftotal.Zone.notnull()] = dftotal[dftotal.Zone.notnull()].shift(periods=1, axis="columns")

In [56]:
# Shift DSL/LTC column info for rows where Facility Name IS null
# (due to structure of the PDF)
# Shift by 3 because we will shift the whole row by 1 to correct facility name next

rows = dftotal[dftotal['Facility Name'].isnull()].index.to_list()
col_mns = ['DSL-3','DSL-4', 'DSL-4-Dementia',
          'LTC-Auxilary Hospital', 'LTC-Nursing Home']

dftotal.loc[rows, col_mns] = dftotal.loc[rows, col_mns].shift(periods=3, axis="columns")

In [57]:
# Shift row information 1 column right where Facility Name IS null
# (due to structure of the PDF)
dftotal[dftotal['Facility Name'].isnull()] = dftotal[dftotal['Facility Name'].isnull()].shift(periods=1, axis="columns")

In [58]:
# Forward fill shifted rows
cols = ['Location','Facility Address', 'Operator Name', 'Operator Type']
dftotal.loc[:,cols] = dftotal.loc[:,cols].ffill()

In [59]:
dftotal.info()
dftotal[dftotal['Location']== 'Slave Lake']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347 entries, 1 to 390
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Zone                   0 non-null      object
 1   Location               347 non-null    object
 2   Facility Name          347 non-null    object
 3   Facility Address       347 non-null    object
 4   Operator Name          347 non-null    object
 5   Operator Type          347 non-null    object
 6   DSL-3                  57 non-null     object
 7   DSL-4                  128 non-null    object
 8   DSL-4-Dementia         89 non-null     object
 9   LTC-Auxilary Hospital  107 non-null    object
 10  LTC-Nursing Home       97 non-null     object
dtypes: object(11)
memory usage: 42.5+ KB


,Zone,Location,Facility Name,Facility Address,Operator Name,Operator Type,DSL-3,DSL-4,DSL-4-Dementia,LTC-Auxilary Hospital,LTC-Nursing Home
377,NaN,Slave Lake,Slave Lake Healthcare Centre,"309 6 Street NE SLAVE LAKE, Alberta T0G 2A2",Alberta Health Services,Alberta Health Services,NaN,NaN,NaN,,NaN
378,NaN,Slave Lake,Vanderwell Heritage Place,"301 6 Avenue NE SLAVE LAKE, Alberta T0G 2A0",Lesser Slave Lake Regional Housing Authority,Private,,NaN,NaN,NaN,NaN


**Designated LTCs Only**

In [48]:
# df_ltc = dftotal[dftotal[['LTC-Auxilary Hospital','LTC-Nursing Home']].notnull().any(1)]

In [49]:
# # Drop zone column
# df_ltc.drop(columns = ['Zone'],inplace = True)
# # Clean symbols
# df_ltc.replace({'': True, np.nan: False}, inplace = True)
# df_ltc.info()
# df_ltc.head()

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


**Export**

In [51]:
# df_ltc.to_csv('../data/alberta/open_alberta_LTC.csv')

In [60]:
# Drop zone column
dftotal.drop(columns = ['Zone'],inplace = True)
# Clean symbols
dftotal.replace({'': True, np.nan: False}, inplace = True)
dftotal.to_csv('../data/alberta/open_alberta_scrape.csv')